# Đồ án cuối kỳ môn khoa học dữ liệu CQ2016/2

## Đề tài: Phân loại danh mục của sản phẩm dựa trên tên sản phẩm

## <font color='blue'> Chi tiết quy trình thu thập dữ liệu, phân tích, huấn luyện dữ liệu để đưa ra model máy học dự đoán danh mục sản phẩm được chia làm những bước chính sau đây </font>

## Mục lục trình bày các bước:
* [B1: Thu thập dữ liệu  ](#first-bullet)
* [B2: Tiền xử lý dữ liệu](#second-bullet)
* [B3: Chuyển data dạng text sang vector](#second-bullet)
* [B4: Áp dụng các mô hình máy học khác nhau để huấn luyện dữ liệu, chọn ra mô hình tối ưu nhất](#second-bullet)

## Import các thư viện cần thiết

In [ ]:
from requests_html import HTMLSession
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re  # For preprocessing
from IPython.display import display, HTML
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

## B1: Thu thập dữ liệu  <a class="anchor" id="first-bullet"></a>

Nhóm thực hiện thu thập dữ liệu trên trang [tiki.vn](https://tiki.vn/). Trước khi thu thập nhóm đã kiểm tra file `robot.txt` và thấy dữ liệu thu thập được hoàn toàn được trang tiki cho phép và hợp lệ

In [ ]:
session = HTMLSession()
r = session.get('https://tiki.vn/')
r.encoding = 'utf-8'

data = []
link_category_dict = dict()

def parse_link(href, currentPage = 1):
    page = href+"&page="+str(currentPage)
    _session = HTMLSession()
    _r = _session.get(page)
    items = _r.html.find('.content .title')

    if len(items) == 0:
        return
    for item in items:
        category = link_category_dict[href]
        data.append([item.text.strip().rstrip("."), category])
    nextPage = currentPage+1
    try:
        print("page ", page)
        parse_link(href, nextPage)
    except Exception as e:
        print("some errors occured", str(e))

def get_categories():
    categories = []
    item_list = r.html.find('li.MenuItem-tii3xq-0 ')
    for item in item_list:
        href = item.find('a', first=True).attrs["href"]
        category = item.text
        categories.append((href, category))
        link_category_dict[href] = category
    return categories

def crawl_data():
    category_tuples = get_categories()
    for category_tuple in category_tuples:
        (href, category_name) = category_tuple
        # Không cần sleep vì tiki.vn đã có cơ chế rate limit request
        parse_link(href)

def save_data():
    np.random.shuffle(data)
    if len(data) != 0:
        with open('product.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["product_title", "category"])
            for item in data:
                writer.writerow(item)

category_tuples = get_categories()
categorydf = pd.DataFrame(list(category_tuples), columns=['đường dẫn', 'tên danh mục'])
display(categorydf)
def run():
    print("start crawling")
    crawl_data()
    save_data()

# uncomment run() to start crawling data
# run()

Sau khi thực hiện lấy danh sách các link danh mục sản phẩm như trên ta tiến hành thu thập dữ liệu đối với từng link. Crawler sẽ crawl hết tất cả các trang đối với từng danh mục bằng cách sử dungj param `page=` truyền vào request url sẽ có dạng như sau

https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner&_lc=Vk4wMzkwMTIwMDQ%3D?page=1

Sau khi crawl dữ liệu thành công ta đọc dữ liệu từ file `product.csv` và tạo data frame bằng pandas. Ta được mẫu dữ liệu như sau

In [ ]:
df = pd.read_csv('product.csv')
print("count = ", df.count())
df.head(10)

Tiến hành thống kê số lượng sản phẩm cho mỗi danh mục sau khi thu thập được

In [ ]:
fig = plt.figure(figsize=(8,6))
colors = ['cyan','green','orange','blue','red','grey','purple','black','grey',
    'turquoise','violet','navy','darkblue']
df.groupby('category').product_title.count().sort_values().plot.barh(
    ylim=0, color=colors, title='Số sản phẩm ứng với mỗi danh mục')
plt.xlabel('Số sản phẩm', fontsize=15)

## B2: Tiền xử lý dữ liệu

Sau khi dữ liệu được thu thập, chúng ta cần xử lý dữ liệu để phục vụ việc training dữ liệu được tốt hơn
Chẳng hạn 1 dòng dữ liệu có `title` như sau

In [ ]:
df.loc[142177]["product_title"]

Ở ví dụ trên, ta thấy có những thông tin thừa thải không cần được đưa vào mô hình để huấn luyện như là các chữ `Combo`, `4`, `(150g)`. Vì vậy công việc tiền xử lý sẽ loại bỏ những thông tin này
Với ví dụ trên, ta mong muốn qua bước tiền xử lý. Thông tin còn lại sẽ là:

Combo 4 hộp Gà Hầm Vissan (150g) => <font color='red'>hộp Gà Hầm Vissan</font>

### Lấy danh sách các stop word

In [ ]:
stop_words = list(line.strip() for line in open('stopword.txt'))
print(stop_words)

Tạo class `TextPreprocessor` implement interface `transformer` của Sklearn. TextPreprocessor sẽ làm nhiệm vụ lowercase, xoá các stopword, số, các kí tự đặc biệt, dấu chấm câu khỏi product_title

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words = []):
        self.stop_words = stop_words
    def fit(self, X_df, y=None):
        categories = X_df['category'].unique()
        # Thêm cột category_id, map category thành số 
        self.category_to_id = {}
        assign_id = 0
        for category in categories:
            self.category_to_id[category] = assign_id
            assign_id += 1  # Get a new id for new author
        # Dict lưu key: id, value: tên category    
        self.id_to_category = {v: k for k, v in self.category_to_id.items()}
        
    def id_to_categoryname(self, category_id):
        return self.id_to_category[category_id]
    
    def get_category_id(self, category):
        return self.category_to_id[category]
    
    def cleanText(self, text): 
        # Convert text to lower
        text = text.lower()
        # Removing non alphabetic words
        # word contains number | hyphen | paranthesis
        text = re.sub(r'\S*\d\S*|-|\(.*\)', r'', text)

        # Removing all the stopwords
        filtered_words = [word for word in text.split() if word not in stop_words]
        text = " ".join(filtered_words)

        # strip multiple spaces
        text = re.sub(' +', ' ', text)

        # strip punctuation
        text = re.sub(r'[^\w\s]', '', text)
        return text

    def transform(self, X_df, y=None):
        new_X_df = X_df.copy()
        new_X_df['category_id'] = new_X_df['category'].map(self.get_category_id)
        new_X_df.drop(['category'], axis=1)
        new_X_df["product_title"] = new_X_df["product_title"].map(self.cleanText)        
        return new_X_df
    

    


### Dữ liệu trước khi tiền xử lý

In [ ]:
df.head(10)

### Dữ liệu sau tiền xử lý

In [ ]:
textProcessor = TextPreprocessor(stop_words)
textProcessor.fit(df)
transformed_df = textProcessor.transform(df)
transformed_df.head(10)

In [ ]:
transformed_df.loc[142177]["product_title"]

vậy là từ `Combo 4 hộp Gà Hầm Vissan (150g)`, dữ liệu đã được tiền xử lý và trở thành `hộp gà hầm vissan`

## Bước 3: Chuyển data dạng text sang vector

### Tách tập huấn huấn luyện, tập test từ data được tiền xử lý

In [ ]:
# Tách tập (train + validation) và tập test theo tỉ lệ 70%:30%

X_trainVal, X_test, y_trainVal, y_test = train_test_split(
    transformed_df['product_title'], transformed_df['category_id'], test_size=0.33, random_state=42)

# tách tập thành tập train và validation từ tập trainVal
X_train, X_val, y_train, y_val = train_test_split(X_trainVal, y_trainVal, test_size=0.33, random_state=1)

### Chuyển data text sang vector số

In [ ]:
count_vect = CountVectorizer(analyzer= "word", stop_words=stop_words)
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

### Transform dữ liệu text của tập validation sang vector từ model đã thu được trươc đó


In [ ]:
X_val_counts = count_vect.transform(X_val)
X_val_tfidf = tfidf_transformer.transform(X_val_counts)

# B4: Dùng các model máy học khác nhau để dự đoán

## Tạo model dự đoán naive bayes

In [ ]:
mnb_clf = MultinomialNB().fit(X_train_tfidf, y_train)

## Dùng model dự đoán tập validation

In [ ]:
multinomialNB_predict = mnb_clf.predict(X_val_tfidf)
multinomialNB_score = accuracy_score(y_val, multinomialNB_predict) * 100
print("Naive bayes Accuracy Score -> ", multinomialNB_score)

## Dùng model Linear support vector machine (SGD)

In [ ]:
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3,
                    random_state=42, max_iter=5, tol=None).fit(X_train_tfidf, y_train)

# predict the labels on validation dataset
sdg_predict = sgd_clf.predict(X_val_tfidf)
sgd_score = accuracy_score(y_val, sdg_predict) * 100
print("SGD Accuracy Score -> ", sgd_score)

## Dùng model Logistic Regression

In [ ]:
lgt_clf = LogisticRegression(n_jobs=1, C=1e5, max_iter=100, solver='lbfgs', multi_class='auto').fit(X_train_tfidf, y_train)
predictions_LGT = lgt_clf.predict(X_val_tfidf)
lgt_score = accuracy_score(y_val, predictions_LGT) * 100
print("LGT Accuracy Score -> ", lgt_score )

In [ ]:
objects = ('Naive Bayes', 'SGD', 'Logistic Regression')
y_pos = np.arange(len(objects))
scores = [multinomialNB_score,sgd_score, lgt_score]

plt.bar(y_pos, scores, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Score')
plt.title('Score for text classfication prediction models')

plt.show()

Sau khi chạy xong 3 mô hình máy học khác nhau thì ta thấy mô hình logistic regression có kết quả tốt nhất.

Ta dùng model logistic regression để thực hiện train trên tập dữ liệu gồm có tập train và tập validation để tối ưu hóa kết quả

In [ ]:
# X_trainVal là tập dữ liệu gồm tạp train và validation
X_counts = count_vect.transform(X_trainVal)
X_tfidf = tfidf_transformer.transform(X_counts)

lr = LogisticRegression(n_jobs=1, C=1e5, max_iter=100, solver='lbfgs', multi_class='auto')
last_lr_clf = lr.fit(X_tfidf, y_trainVal)

X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)


last_lr_predict = last_lr_clf.predict(X_test_tfidf)
last_lr_test_score = accuracy_score(y_test, last_lr_predict) * 100
print("Logistic regression test Accuracy Score -> ", last_lr_test_score)

Sau khi dự đoán với tập test ta thu được score: 85.01

## Dự đoán với dữ liệu bất kì

In [ ]:
X_input = ["Bàn phím acer", "lược sử loài người", "Trái đất hình thành như thế nào?",
             "Như là giấc mơ", "Găng tay xe đạp", "Bình nước nóng sanyo"]
X_input_counts = count_vect.transform(X_input)
X_input_tfidf = tfidf_transformer.transform(X_input_counts)
x_input_predict= last_lr_clf.predict(X_input_tfidf)
counter = 0
for doc, category_id in zip(X_input, x_input_predict):
    print('%r => %s' % (doc, textProcessor.id_to_categoryname(category_id)))
    if(counter == 10):
        break
    counter += 1